if you find this feature or notebook has anything helpful or unique
    
please do <h4>upvote!</h4>
    
> other notebooks from me :

> check prev feature and description [here](https://www.kaggle.com/code/tyrionlannisterlzy/jpx-lgbm-model-auto-search-para)
   
> check means to auto search parameters [here](https://www.kaggle.com/code/tyrionlannisterlzy/jpx-early-template-for-everyone-to-use)

<h1>Importing the Library

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from sklearn.tree import DecisionTreeRegressor

<h3>concat function

In [ ]:
def concat_df(df1, df2):
    df1 = pd.concat([df1, df2],
                    ignore_index=True, sort=False
                    ).drop_duplicates(["RowId"], keep="first")
    return df1

<h1>Loading the dataset

In [ ]:
path = "../input/jpx-tokyo-stock-exchange-prediction/"
df_prices = pd.read_csv(f"{path}train_files/stock_prices.csv")
df_prices = df_prices[~df_prices["Target"].isnull()]
prices = pd.read_csv(f"{path}supplemental_files/stock_prices.csv")
df_prices = concat_df(df_prices, prices)
df_prices = df_prices[df_prices.Date>="2021-08-07"]
df_prices.info(show_counts=True)

In [ ]:
df_prices

<h1>Trying New Feature
    

so here the shift in the same day may give us some insights into the overall market sitaution

because its related to other exchanges.

near the bordery of each day,we still have 3 days (1 , 2 ,5 day after this exchange)

and I used the np.log() function to downsize the impact of them , due to its scale

any problems if I tried so?

In [ ]:
def prep_prices(prices):
    prices.fillna(0,inplace=True)
    prices['High_shift1']=prices['High'].shift(-1)
    prices['Low_shift1'] =prices['Low'].shift(-1)
    prices['Open_shift1']=prices['Open'].shift(-1)
    prices['Close_shift1']=prices['Close'].shift(-1)
    prices['Volume_shift1']=prices['Volume'].shift(-1)
    
    prices['High_shift2']=prices['High'].shift(-2)
    prices['Low_shift2'] =prices['Low'].shift(-2)
    prices['Open_shift2']=prices['Open'].shift(-2)
    prices['Close_shift2']=prices['Close'].shift(-2)
    prices['Volume_shift2']=prices['Volume'].shift(-2)
    
    prices['High_shift3']=prices['High'].shift(-3)
    prices['Low_shift3'] =prices['Low'].shift(-3)
    prices['Open_shift3']=prices['Open'].shift(-3)
    prices['Close_shift3']=prices['Close'].shift(-3)
    prices['Volume_shift3']=prices['Volume'].shift(-3)
    
#     prices['High_shift4']=prices['High'].shift(-4)
#     prices['Low_shift4'] =prices['Low'].shift(-4)
#     prices['Open_shift4']=prices['Open'].shift(-4)
#     prices['Close_shift4']=prices['Close'].shift(-4)
#     prices['Volume_shift4']=prices['Volume'].shift(-4)
    
    prices['High_shift+1']=prices['High'].shift(1)
    prices['Low_shift+1'] =prices['Low'].shift(1)
    prices['Open_shift+1']=prices['Open'].shift(1)
    prices['Close_shift+1']=prices['Close'].shift(1)
    prices['Volume_shift+1']=prices['Volume'].shift(1)
    
    prices['High_shift+2']=prices['High'].shift(2)
    prices['Low_shift+2'] =prices['Low'].shift(2)
    prices['Open_shift+2']=prices['Open'].shift(2)
    prices['Close_shift+2']=prices['Close'].shift(2)
    prices['Volume_shift+2']=prices['Volume'].shift(2)
    
    prices['High_shift+3']=prices['High'].shift(3)
    prices['Low_shift+3'] =prices['Low'].shift(3)
    prices['Open_shift+3']=prices['Open'].shift(3)
    prices['Close_shift+3']=prices['Close'].shift(3)
    prices['Volume_shift+3']=prices['Volume'].shift(3)
    
#     prices['High_shift+4']=prices['High'].shift(4)
#     prices['Low_shift+4'] =prices['Low'].shift(4)
#     prices['Open_shift+4']=prices['Open'].shift(4)
#     prices['Close_shift+4']=prices['Close'].shift(4)
#     prices['Volume_shift+4']=prices['Volume'].shift(4)

    
    prices['v_avg']=np.log(prices['Volume']*(prices['High']+prices['Low']+prices['Open']+prices['Close'])/4+1)
    prices['v_avg-1']=np.log(prices['Volume_shift1']*(prices['High_shift1']+
                    prices['Low_shift1']+prices['Open_shift1']+prices['Close_shift1'])/4+1)
    prices['v_avg-2']=np.log(prices['Volume_shift2']*(prices['High_shift2']+
                    prices['Low_shift2']+prices['Open_shift2']+prices['Close_shift2'])/4+1)
    prices['v_avg-3']=np.log(prices['Volume_shift3']*(prices['High_shift3']+
                    prices['Low_shift3']+prices['Open_shift3']+prices['Close_shift3'])/4+1)
#     prices['v_avg-4']=np.log(prices['Volume_shift4']*(prices['High_shift4']+
#                     prices['Low_shift4']+prices['Open_shift4']+prices['Close_shift4'])/4+1)

    
    prices['v_avg+1']=np.log(prices['Volume_shift+1']*(prices['High_shift+1']+
                    prices['Low_shift+1']+prices['Open_shift+1']+prices['Close_shift+1'])/4+1)
    prices['v_avg+2']=np.log(prices['Volume_shift+2']*(prices['High_shift+2']+
                    prices['Low_shift+2']+prices['Open_shift+2']+prices['Close_shift+2'])/4+1)
    prices['v_avg+3']=np.log(prices['Volume_shift+3']*(prices['High_shift+3']+
                    prices['Low_shift+3']+prices['Open_shift+3']+prices['Close_shift+3'])/4+1)
#     prices['v_avg+4']=np.log(prices['Volume_shift+4']*(prices['High_shift+4']+
#                     prices['Low_shift+4']+prices['Open_shift+4']+prices['Close_shift+4'])/4+1)
    prices.fillna(0,inplace=True)
    return prices

<h2>UPdated and shifted reversely </h2>

    
    
<h2>got me LB 5.978!</h2>

<h3>I deleted the open  close and high and low col here

In [ ]:
df_prices = prep_prices(df_prices)

# feats=["SecuritiesCode","Open","High","Low","Close","Volume","v_avg","avg","AdjustmentFactor","ExpectedDividend","SupervisionFlag"]
feats=["SecuritiesCode","v_avg","v_avg-1","v_avg-2","v_avg-3","v_avg+1","v_avg+2","v_avg+3","AdjustmentFactor","ExpectedDividend","SupervisionFlag"]
model = DecisionTreeRegressor()
model.fit(df_prices[feats],df_prices["Target"])
model.score(df_prices[feats],df_prices["Target"])

<h2>Make submission

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices,options,financials,trades,secondary_prices,sample_prediction) in iter_test:
    prices = prep_prices(prices)
#     prices['v_avg']=np.log(prices['Volume']*(prices['High']+prices['Low']+prices['Open']+prices['Close'])/4+1)
#     prices['avg']=np.log((prices['High']+prices['Low']+prices['Open']+prices['Close'])/4+1)
    prices["Target"] = model.predict(prices[feats])
    if prices["Volume"].min()==0:
        sample_prediction["Prediction"] = prices["Target"]/prices["Volume"]
    else:
        sample_prediction["Prediction"] = prices["Target"]
    sample_prediction.sort_values(by = "Prediction", ascending=False, inplace=True)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction.sort_values(by = "SecuritiesCode", ascending=True, inplace=True)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)